In [3]:
from selenium import webdriver
import pandas as pd
import bs4 as bs
import urllib.request
import os
from pandas import ExcelFile
import gender_guesser.detector as gender

In [6]:
driver = webdriver.Firefox()
driver.get("http://dlt.aals.org")#put the adress of your page here

In [7]:
username = 'mlivermore@virginia.edu'
password = 'X33y22Z11'
driver.find_element_by_xpath('//*[@id="user_username"]').send_keys(username);
driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password);
driver.find_element_by_xpath('/html/body/div/div/div/form/input[3]').click()

In [8]:
def search_school(school_name):
    driver.get('https://dlt.aals.org/dlts')
    elem = driver.find_element_by_xpath("/html/body/div/div/div/form/div[1]/div[1]/div/div/a/span")
    elem.click()
    driver.find_element_by_xpath("/html/body/div/div/div/form/div[1]/div[1]/div/div/div/div/input").send_keys(school_name)
    driver.find_element_by_xpath("/html/body/div/div/div/form/div[1]/div[1]/div/div/div/ul/li").click()
    example = driver.find_element_by_xpath('/html/body/div/div/div/form/div[1]/div[1]/div/div/div/ul/li')
    if example.text[:16] == 'No results match':
        school_exists = False
    else:
        school_exists = True
        driver.find_element_by_xpath('//*[@id="q_tenured_and_dean"]').click()
        driver.find_element_by_xpath("/html/body/div/div/div/form/div[2]/div/input").click()
    return school_exists

In [29]:
search_school('university of alabama')

True

In [9]:
def get_page_data(html, cur_data):
    dfs = pd.read_html(html)
    data = dfs[0].iloc[:,:-1]
    cur_data = pd.concat([cur_data, data])
    return cur_data
def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

In [19]:
#New List
df = pd.read_excel('school_list_update.xlsx', sheet_name='Sheet1')
list_schools = list(df['Moved From'])

In [20]:
cur_data = pd.DataFrame() 
for school in list_schools:
    school_exists = search_school(school)
    if not school_exists and ',' in school:
        school_exists = search_school(school.split(', ')[0])
    elif not school_exists and 'at' in school:
        school_exists = search_school(school.split(' at ')[0])
    if not school_exists:
        school_exists = search_school('The ' + school)
    if not school_exists and ' University' in school:
        school_exists = search_school(school.split(' University')[0])
    if school_exists:   
        html = driver.page_source
        cur_data = get_page_data(html, cur_data)
        soup = bs.BeautifulSoup(html, 'lxml')
        next_page_url = soup.find('a', {'rel': 'next'})

        while next_page_url:
            next_link = next_page_url.get('href')
            link = 'https://dlt.aals.org' + next_link
            driver.get(link)
            html = driver.page_source
            cur_data = get_page_data(html, cur_data)
            soup = bs.BeautifulSoup(html, 'lxml')
            next_page_url = soup.find('a', {'rel': 'next'})
        cur_data.reset_index(inplace = True, drop = True)
    else:
        print('{} is not in the database'.format(school))
cur_data.to_csv(create_path('school_data', 'full_data_new_update.csv'))

In [21]:
#Add Gender information
d = gender.Detector()
path = os.path.join(os.getcwd(), 'school_data', 'full_data_new_update.csv')
df = pd.read_csv(path, index_col=[0])
gender_dict = {'male': 1, 'mostly_male': 1, 'female': 0, 'mostly_female': 0, 'unknown': 2, 'andy': 2}
genders = [d.get_gender(name) for name in df['First Name']]
labels_gender = [gender_dict[gen] for gen in genders]
df['Gender'] = genders
df['Gender Label'] = labels_gender
df.to_csv(os.path.join(os.getcwd(), 'school_data', 'full_data_new_update_with_gender.csv'))

In [17]:
list_schools

<_io.TextIOWrapper name='list_schools.txt' mode='r' encoding='cp1252'>

After you have logged in, Selenium is able to navigate to any webpage. We will navigate to the pages on the list:

In [4]:
faculty_list = pd.read_csv('UVa_law_faculty.csv')
faculty_list
faculty_list.drop(columns = ['Unnamed: 0'], inplace=True)
faculty_list

,Resident
0,Kenneth S. Abraham
1,Barbara E. Armacost
2,Aditya Bamzai
3,Charles Barzun
4,Michal Barzuza
5,Richard J. Bonnie
6,Josh Bowers
7,Maureen E. Brady
8,Stephen Braga
9,Darryl K. Brown


In [86]:
def to_float_or_int(input_list):
    new_list = []
    for x in input_list:
        x = x.replace(',','')
        try:
            value = int(x)
        except ValueError:
            try:
                value = float(x)
            except:
                value = ''
        new_list.append(value)
    return new_list
new_list = to_float_or_int(column_1)
print(column_1)
print(new_list)

['82', '1,594', ' 732', ' 466', ' 74', '\xa0', '26.13', ' 58 ']
[82, 1594, 732, 466, 74, '', 26.13, 58]


In [ ]:
full_name = faculty_list.Resident[0]
first_name = full_name.split(' ')[0]
middle_name = full_name.split(' ')[1]
last_name = full_name.split(' ')[2]

link = 'https://heinonline-org.proxy01.its.virginia.edu/HOL/AuthorProfile?action=edit&search_name=' + last_name + '%2C%20' + first_name + '%20' + middle_name + '&collection=journals'
driver.get(link)